In [1]:
import os
import re
import warnings

import pandas as pd
import numpy as np
import torch

from transformers import (
    AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, 
    DataCollatorForSeq2Seq, 
)

from datasets import load_metric, Dataset

import wandb
import nltk

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jake/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()
NGPU, NCPU

(1, 16)

# Paths and Names

In [3]:
### paths and names

PROJECT_NAME = 'news-topic-keyphrase-generation-model-dev'
RUN_ID = 'v1'

DATA_PATH = './data/total_2513_samples.pickle'

MODEL_CHECKPOINT = 'paust/pko-t5-base'
METRIC_NAME = 'rouge'

NOTEBOOK_NAME = './train.ipynb'

ROOT_PATH = './'
SAVE_PATH = os.path.join(ROOT_PATH, '.log')

model_name = re.sub(r'[/-]', r'_', MODEL_CHECKPOINT).lower()
run_name = f'{model_name}_{RUN_ID}'
output_dir = os.path.join(SAVE_PATH, run_name)

!mkdir -p {SAVE_PATH}

In [4]:
%env WANDB_PROJECT={PROJECT_NAME}
%env WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find {NOTEBOOK_PATH}.


env: WANDB_PROJECT=news-topic-keyphrase-generation-model-dev
env: WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all


wandb: Currently logged in as: dotsnangles. Use `wandb login --relogin` to force relogin


True

# Training Args

In [5]:
report_to="wandb"

num_train_epochs = 30
per_device_train_batch_size = 2
per_device_eval_batch_size = 2
gradient_accumulation_steps = 1

optim = 'adamw_torch' # 'adamw_torch' or 'adamw_hf'

learning_rate = 3e-6 * NGPU
weight_decay = 0.01
adam_epsilon = 1e-8

lr_scheduler_type = 'cosine' # 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', 'constant', 'constant_with_warmup'
warmup_ratio = 0

save_total_limit = 2

load_best_model_at_end = True
metric_for_best_model = 'eval_loss'

save_strategy = "epoch"
evaluation_strategy = "epoch"

logging_strategy = "steps"
logging_first_step = True 
logging_steps = int(500 / NGPU)

predict_with_generate=True
generation_max_length=64
# generation_num_beams=5

fp16 = False

# Model & Tokenizer & Metric

- 모델과 토크나이저, 그리고 평가지표를 계산하는 데 사용할 함수를 불러옵니다.
- 모델의 config에는 사용하지 않는 설정이 포함되어 있습니다. 삭제합니다.

In [6]:
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, config=config)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
metric = load_metric(METRIC_NAME)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Functions

- 모델 훈련을 위해 전처리를 수행해주는 함수를 선언합니다.
- 입력 문장이 input이 되고 요약문이 label이 됩니다.
- tokenizer를 사용해 input_ids로 변환하고 입력값에는 attention_mask를 생성해줍니다.
- 입력 문장에 prefix를 추가하여 성능 실험을 해볼 수 있으나 지금은 적용하지 않겠습니다.

In [8]:
prefix = "key phrase generation: "

max_input_length = 512
max_target_length = 64

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["input_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

- 모델의 예측값과 위에서 전처리한 라벨을 활용하여 평가지표를 출력하는 함수를 선언합니다.
- ROUGE를 평가지표로 사용합니다.
    - ROUGE-N (N-gram) scoring
    - ROUGE-L (Longest Common Subsequence) scoring
        - sentence-level: Compute longest common subsequence (LCS) between two pieces of text. Newlines are ignored. This is called rougeL in this package.
        - summary-level: Newlines in the text are interpreted as sentence boundaries, and the LCS is computed between each pair of reference and candidate sentences, and something called union-LCS is computed. This is called rougeLsum in this package.

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# Inputs and Labels

- csv로 저장된 데이터를 불러와 9:1로 훈련과 검증에 사용합니다.
- 선언한 전처리 함수를 적용하여 데이터세트를 생성합니다.

In [10]:
data_df = pd.read_pickle(DATA_PATH)

In [11]:
data_df

,input_text,target_text,prefix
0,다음 달부터 15억 초과 주택 대출 허용…ltv 50%도 다음 달부터는 무주택자의 ...,"주택담보대출,ltv 규제 완화,dsr",extract keywords:
1,"송혜교x장기용 ""끝까지 사랑일 거야"" ..애절함 폭발 '지헤중', 순간 최고 9% ...","송혜교,장기용,사랑,애절함,지헤중,최고 시청률,민여사,강정자,윤수완,신유정",extract keywords:
2,"1개당 3만원? 자가검사키트 막판 사재기...정부 ""17일 이후 해소"" 사진=온라인...",자가검사키트,extract keywords:
3,돌싱글즈 유세윤 어릴 적 부모님 이혼사실 몰라.. 父해외 출장 간 줄 [스타뉴스 공...,"돌싱글즈,유세윤,부모님 이혼,해외 출장,빈하영,정윤식",extract keywords:
4,"내릴 땐 '찔끔', 올릴 땐 '확'…휘발유 가격, 급속도로 인상","가격 인상,휘발유 가격",extract keywords:
...,...,...,...
392,"T1 잔류 선택한 '페이커' 이상혁 ""내가 돌아왔다"" '페이커' 이상혁이 T1에 잔...","잔류,이상혁,페이커,T1",extract keywords:
393,영국 철도노조 30여년 만에 최대규모 파업…기차역 텅 비어 임금인상·구조조정 중단 ...,"영국철도노조,최대규모 파업,임금인상,구조조정 중단요구",extract keywords:
394,‘밥심’ 마리아 “韓 아이돌 데뷔가 꿈→30kg 감량” 매경닷컴 MK스포츠 손진아 ...,"밥심,마리아,한국 아이돌,데뷔,감량,30kg,미스트롯2,올 하트,외국인 참가자,SB...",extract keywords:
395,손연정 '멀리 멀리' 24일 경기도 안산 아일랜드 CC에서 열린 한국여자프로골프 ...,"한국여자프로골프,엘크루-TV조선 프로 셀러브리티 2021,손연정",extract keywords:


In [12]:
dataset = Dataset.from_pandas(data_df).shuffle(seed=100).train_test_split(0.2)
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [13]:
train_dataset = train_dataset.map(preprocess_function, 
                                  batched=True, 
                                  num_proc=NCPU, 
                                  remove_columns=train_dataset.column_names)

eval_dataset = eval_dataset.map(preprocess_function, 
                                batched=True, 
                                num_proc=NCPU, 
                                remove_columns=eval_dataset.column_names)
print(train_dataset)
print(eval_dataset)

Map (num_proc=16):   0%|          | 0/2010 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/503 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2010
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 503
})


In [14]:
tokenizer.decode(train_dataset['input_ids'][0])

'key phrase generation: 임영웅, \'다시 만날 수 있을까\'로 엠카 1위...영웅시대 감사합니다 [ten★] [텐아시아=조준원 기자]가수 임영웅이 근황을 전했다. 임영웅은 12일 임영웅의 공식 인스타그램에는 "아티스트 임영웅 mnet ‘엠카운트다운’ 1위, 영웅시대 여러분, 감사합니다"라는 글과 함께 두 장의 사진을 게재했다. 공개된 사진 속 mnet \'엠카운트다운\'에서 1위를 차지한 뒤 트로피 인증샷을 남기고 있는 임영웅의 모습이 담겨 있다. 한편 임영웅은 2일 정규 1집 \'im hero\'(아임 히어로)를 발매했다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [15]:
tokenizer.decode(train_dataset['labels'][0])

'임영웅,엠카운트다운,영웅시대,아임 히어로 발매</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

# Train

- 상단에서 선언한 HP와 각종 변수들로 training_args를 생성합니다.

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    run_name=run_name,
    report_to=report_to,

    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    optim=optim,

    learning_rate=learning_rate,
    weight_decay=weight_decay,
    adam_epsilon=adam_epsilon,

    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,

    save_total_limit=save_total_limit,

    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,

    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,

    logging_strategy=logging_strategy,
    logging_first_step=logging_first_step, 
    logging_steps=logging_steps,

    predict_with_generate=predict_with_generate,
    generation_max_length=generation_max_length,
    # generation_num_beams=generation_num_beams,

    fp16=fp16,
)

- 배치 단위로 padding처리를 하여 훈련 속도를 높히기 위해 data_collator를 선언합니다.
- 생성한 객체들을 trainer에 포함시켜 훈련 준비를 마칩니다.

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

trainer = Seq2SeqTrainer(
    model=model,
    
    args=training_args,
    
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    
    tokenizer=tokenizer,
    data_collator=data_collator,
    
    compute_metrics=compute_metrics,
)

- 훈련을 시작합니다.

In [18]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,4.231000,1.594958,9.254600,1.445900,9.075700,9.119800,54.954300
2,0.813600,0.557981,15.793800,2.696400,15.354300,15.430300,31.111300
3,0.632800,0.486895,17.914000,3.757100,17.416400,17.478400,32.566600
4,0.577300,0.466830,18.716100,3.943600,18.255200,18.255800,31.188900
5,0.516700,0.453396,17.434200,3.406100,16.872000,16.834700,31.334000
6,0.524500,0.443857,19.319400,4.046600,18.817600,18.768200,30.198800
7,0.493700,0.436659,19.514000,3.820100,19.010000,19.020400,28.180900
8,0.488100,0.434352,19.947800,3.819600,19.396800,19.427000,29.270400
9,0.479400,0.432945,20.289800,4.242100,19.935500,19.955200,28.612300
10,0.467900,0.428887,20.183600,3.611800,19.598500,19.641300,26.697800


TrainOutput(global_step=30150, training_loss=0.7454467680442393, metrics={'train_runtime': 17408.7047, 'train_samples_per_second': 3.464, 'train_steps_per_second': 1.732, 'total_flos': 4.38843693072384e+16, 'train_loss': 0.7454467680442393, 'epoch': 30.0})

- 옵티마이저 스테이트를 포함한 훈련 재개에 사용할 파일들을 삭제해줍니다.
- 추후 불러올 모델과 토크나이저 관련 파일만 남겨둡니다.

In [19]:
# keep = [
#     'added_tokens.json',
#     'config.json',
#     'pytorch_model.bin',
#     'special_tokens_map.json',
#     'tokenizer.json',
#     'tokenizer_config.json',
#     'vocab.txt'
# ]

# ckpts = os.listdir(output_dir)
# for ckpt in ckpts:
#     ckpt = os.path.join(output_dir, ckpt)
#     for item in os.listdir(ckpt):
#         if item not in keep:
#             os.remove(os.path.join(ckpt, item))

# Generate

- 훈련을 마치면 Evaluation Loss 기준 Best 모델이 로드되어 있습니다.
- trainer는 Greedy Search를 수행하도록 설정되어 있습니다.
- Evaluation 데이터를 활용해 모델의 출력을 간단히 살펴봅니다.

In [20]:
preds = trainer.predict(eval_dataset)

In [21]:
for data, pred in zip(eval_dataset, preds.predictions):
    context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)
    summary = tokenizer.decode(data['labels'], skip_special_tokens=True)
    pred = tokenizer.decode(pred[2:], skip_special_tokens=True)
    # print(f'입력: {context}')
    print(f'정답: {summary}')
    print(f'예측: {pred}', end='\n\n')

정답: 에어로스페이스 콘퍼런스,이성용 공군참모총창,국회 국방위원,첨단 과학기술,전시 장비
예측: ,에어로스페이스 콘퍼런스,이성용 공군참모총장,서울 공군회관,첨단과학기술,전시장비

정답: 이은지,제주종합경기장,실내수영장,2021 경영 국가대표 선발대회,여자 배영 50m 결승,출발,기록: 28초52
예측: 지,힘찬 출발

정답: 일상,노홍철,와인 시음회,인스타그램
예측: ,와인 시음회,럭셔리한 일상,인스타그램

정답: 레고랜드 주변,컨벤션 건립 예산안,도의회 상임위 통과,강원국제전시컨벤션센터,제2회 추가경정예산안,원안 가결,토지 매입비,임시주차장,공사비,예산결산특별위원회,본회의,최종 확정
예측: 랜드,컨벤션 건립 예산안,도의회 상임위 통과,강원국제전시컨벤션센터,건립 예산안,도의회 상임위 통과,강원도의회 경제건설위원회,제2회 추가경정예산안,강원도 글로벌투자통상국,제2회 추가경정

정답: 코로나19,확진,이승기
예측: ,코로나19 확진,일반 감기 증상,걱정 감사

정답: 수베로 감독,고척스카이돔,2021 KBO리그,한화 이글스,키움 히어로즈,3연패 탈출,선발 김민우,노시환 3타점,힐리 2타점,김민하 희생타 1타점,6-1 승리,최재훈
예측: 마님,수베로 감독,키움 히어로즈,3연패,탈출,한화 이글스,키움 히어로즈,한화 이글스,한화 이글스,키움 히어로즈,2연패,3연승,3연패 탈출,최재훈,노시환,노시환,김민하,희생

정답: 글로벌 마켓,뉴욕증시,크리스마스,급락
예측: 증시,산타,급락

정답: 신진서,신민준,도전 5번기,한국기원
예측: 서,신민준,도전 5번기,한국기원,도전 5전기,

정답: 아우디 딜러,2021 아우디 딜러 어워즈,탑 아우디언,코로나19,고진모터스,한서모터스,
예측: ,2021 아우디 딜러 어워즈,탑 아우디언,세일즈,A/S,마케팅,베스트 딜러사,베스트 세일즈 딜러사,베스트 딜러사,베스트 마케팅 딜러사,고진모터스,아이언모터스,

정답: 쌀값 폭락,쌀값 안정 대책,쌀값
예측:  폭락 대책,전국 8곳 도지사,정부,쌀값 폭락 대책

정답: 고민시,미성년자 음주 논란,